In [12]:
!pip install -U segmentation-models-pytorch

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.3/121.3 kB 5.1 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=79873e7e22d0893ad8283dae43e269db1526da4f53a3fd4dd23c2a10814777a5
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=02af5f95c3734013aa41f315a280b13a91f30d3ef9e6230801d7fde375c9be06
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch pretrainedmodels


In [1]:

import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import cv2 as cv
from sklearn.model_selection import train_test_split
from torchvision.transforms import ToTensor
import random
from glob import glob

class RetinaDataset(Dataset):
    def __init__(self, images, masks, transform=None):
        self.images = images
        self.masks = masks
        self.to_tensor = ToTensor()
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):

        image = cv.imread(self.images[idx], cv.IMREAD_COLOR)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)

        mean = image.mean(axis=(0, 1))
        std = image.std(axis=(0, 1))
        image = (image - mean[None, None, :]) / std[None, None, :]
        image = image.astype(np.float32)

        mask = cv.imread(self.masks[idx], cv.IMREAD_GRAYSCALE)
        mask[mask > 0] = 1
        mask = mask.astype(np.float32)

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image, mask = augmented["image"], augmented["mask"]

        image_tensor = self.to_tensor(image)
        mask_tensor = self.to_tensor(mask)

        return image_tensor, mask_tensor


def load_data(base_directory="/kaggle/input/retina-blood-vessel/Data", batch_size=4, train_transform=None, test_transform=None):
    
    test_images = sorted(glob(os.path.join(base_directory, "test", "image", "*.png")))
    test_masks = sorted(glob(os.path.join(base_directory, "test", "mask", "*.png")))

    test_dataset = RetinaDataset(test_images, test_masks, transform=test_transform)

   
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    return test_loader

In [17]:
import torch
import torch.nn as nn
def conv_block(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
        nn.ReLU(inplace=True),
    )

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.criterion = nn.BCEWithLogitsLoss()

        self.enc1 = conv_block(in_channels, 64)
        self.enc2 = conv_block(64, 128)
        self.enc3 = conv_block(128, 256)
        self.enc4 = conv_block(256, 512)
        self.enc5 = conv_block(512, 1024)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.upconv4 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.upconv3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)

        self.dec1 = conv_block(1024, 512)
        self.dec2 = conv_block(512, 256)
        self.dec3 = conv_block(256, 128)
        self.dec4 = conv_block(128, 64)

        self.out = nn.Conv2d(64, out_channels, kernel_size=1)

        self.test_loss_sum = 0.0
        self.test_iou_sum = 0.0
        self.test_batches = 0
        self.test_dice_sum = 0.0
        self.test_acc_sum = 0.0

    def forward(self, x):
        x1 = self.enc1(x)
        x2 = self.enc2(self.pool(x1))
        x3 = self.enc3(self.pool(x2))
        x4 = self.enc4(self.pool(x3))
        x5 = self.enc5(self.pool(x4))

        x = self.upconv4(x5)
        x = torch.cat([x, x4], dim=1)
        x = self.dec1(x)

        x = self.upconv3(x)
        x = torch.cat([x, x3], dim=1)
        x = self.dec2(x)

        x = self.upconv2(x)
        x = torch.cat([x, x2], dim=1)
        x = self.dec3(x)

        x = self.upconv1(x)
        x = torch.cat([x, x1], dim=1)
        x = self.dec4(x)

        x = self.out(x)
        return x
    def common_step(self, batch, batch_idx, device="cuda"):
        image, mask = batch
        image = image.to(device)
        mask = mask.to(device)
        pred = self.forward(image)
        loss = self.criterion(pred, mask)
        pred = torch.sigmoid(pred)
        mask = mask.round().long()
        tp, fp, fn, tn = smp.metrics.get_stats(pred, mask, mode="binary", threshold=0.5)  # type: ignore
        iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro").item()
        dice = (2 * tp.sum() / (2 * tp.sum() + fp.sum() + fn.sum())).item()
        acc = smp.metrics.accuracy(tp, fp, fn, tn, reduction="macro").item()
        return loss, iou_score, dice, acc
    
    def test_step(self, batch, batch_idx, device="cuda"):
            loss, iou_score, dice, acc = self.common_step(batch, batch_idx, device="cuda")
            self.test_loss_sum += loss
            self.test_iou_sum += iou_score
            self.test_dice_sum += dice
            self.test_acc_sum += acc
            self.test_batches += 1
            return loss,iou_score, dice, acc
    
    def on_test_epoch_end(self):
            avg_train_loss = self.test_loss_sum / self.test_batches
            avg_train_iou = self.test_iou_sum / self.test_batches
            avg_train_dice = self.test_dice_sum / self.test_batches
            avg_train_acc = self.test_acc_sum / self.test_batches
            self.test_loss_sum = 0.0
            self.test_iou_sum = 0.0
            self.test_dice_sum = 0.0
            self.test_acc_sum = 0.0
            self.test_batches = 0
            return avg_train_loss, avg_train_iou, avg_train_dice, avg_train_acc
    

In [4]:
def set_seed():
    torch.manual_seed(5)
    torch.cuda.manual_seed(5)
    torch.cuda.manual_seed_all(5)
    np.random.seed(5)
    random.seed(5)

In [18]:
import os
import torch
import albumentations as A
import numpy as np
from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp
set_seed()


model = UNet(in_channels=3, out_channels=1)
checkpoint = torch.load("/kaggle/input/best-seg-model/best_iou_seg.pth")
model.load_state_dict(checkpoint)
model.eval()

# Define transformation
test_transform = A.Compose([
    A.Resize(512, 512),
])

def transform_image(image):
    augmented = test_transform(image=np.array(image))
    return augmented["image"]


test_loader = load_data("/kaggle/input/retina-blood-vessel/Data", batch_size=4, test_transform=test_transform)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


def test_model(model, test_loader, device="cuda"):
    model.to(device)
    model.eval()
    torch.set_grad_enabled(False)
    for batch_idx, batch in enumerate(test_loader):
        model.test_step(batch, batch_idx)
    avg_test_loss, avg_test_iou, avg_test_dice, avg_test_acc = model.on_test_epoch_end()
    print(f"Test Loss: {avg_test_loss:.4f}, Test IoU: {avg_test_iou:.4f}, Test Dice: {avg_test_dice:.4f}, Test Accuracy: {avg_test_acc:.4f}")
    return avg_test_loss, avg_test_iou, avg_test_dice, avg_test_acc
    
test_model(model, test_loader)


<ipython-input-18-e1f64621a70f>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load("/kaggle/input/best-seg-model/best_iou_seg.pth")


Test Loss: 0.1743, Test IoU: 0.6959, Test Dice: 0.8205, Test Accuracy: 0.9572


(tensor(0.1743, device='cuda:0'),
 0.6959471046924591,
 0.8205029368400574,
 0.9572273254394531)